# Imports

In [16]:
from datetime import datetime
import pandas as pd
import math

# Reading events into DataFrames

In [19]:
nb_lines = math.inf

clicks_content = []
i = 0
with open('test_results/clicks', 'r') as f:
    for line in f:
        clicks_content.append(eval(line))
        i += 1
        if i == nb_lines:
            break


displays_content = []
i = 0
with open('test_results/displays', 'r') as f:
    for line in f:
        displays_content.append(eval(line))
        i += 1
        if i == nb_lines:
            break

In [20]:
df_clicks = pd.DataFrame(clicks_content)
df_clicks.head()

,eventType,uid,timestamp,ip,impressionId
0,click,57e9e910-07e8-4157-8120-59e269b7e503,1624787312,110.164.61.160,ab2cc313-6fa5-42fe-9622-efd33214b5b9
1,click,f4417a0a-458f-40fd-8441-d1b312f7a24e,1624787312,149.244.126.18,8cf59b59-74ca-452b-be15-0b56cb51398a
2,click,f4417a0a-458f-40fd-8441-d1b312f7a24e,1624787313,45.91.152.60,68c93e16-c8a2-4cfb-b514-c78bd424330c
3,click,32b3f845-0d02-466c-95e7-a9177821541a,1624787313,156.109.123.79,05eeeb06-f2b3-4914-b3d2-6c39a75d81b7
4,click,c1d7d707-3d72-427a-8e04-bd3faabd4208,1624787313,207.52.194.203,517c1546-fe9a-49cb-bff8-27595d272fa1


In [21]:
# time elapsed
datetime.fromtimestamp(df_clicks['timestamp'].max() - df_clicks['timestamp'].min())

datetime.datetime(1970, 1, 1, 3, 50)

# Count of IPs per user

When a uid has too many IP addresses, it either means that the user is connected to a lot of devices and each device is connected to a different network (very unlikely for more than 2-3 IPs), or, it's a fraud.

In [27]:
#We notice that more than 75% of users only have one IP address, which is logical. The max is 649, which is not normal. 

df_clicks_ips_per_user = df_clicks.groupby(['uid']).count()['ip']
df_clicks_ips_per_user.describe()

count    19030.000000
mean         2.569469
std         30.529604
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        649.000000
Name: ip, dtype: float64

In [30]:
# Finding the maximum allowed number of Ips per user (3 seems to be a good choice)

df_clicks_ips_per_user[df_clicks_ips_per_user > 3]

uid
01aa1f62-fd0c-40a1-a7cf-8e6adc1b4c95    639
07c38e89-9750-4727-8721-b80b5b03db4e    571
0d878df4-9966-43c8-aeb1-8c736bfbf772    624
10ffe29f-21b7-4f96-9540-91e77ee17d20    598
141a7059-895c-4f74-a002-d1b7e5e6cc85    598
166aeaf4-9995-475a-8a50-f0ca9a34160e    597
26c91786-7eee-46df-91cc-59eeae4350d2    631
32b3f845-0d02-466c-95e7-a9177821541a    592
3b72893e-916b-406d-9979-1f55c4a36678    625
3bf12f77-5084-42b1-a9f4-5b59c8c3ba12    617
438bb141-9d7b-40ec-8a4a-90f220610e63    595
469bbf16-fb45-4df2-9cac-bf0069980a2d    597
48d35610-7b97-4ce6-9ca4-725c512409da    591
4bce2c20-3fed-4af1-9177-542ee232ece2    583
57053291-e5e0-48f3-b1ed-880725ad44d0    591
57529d29-20b5-40bf-b8d8-d514462b381b    599
57e9e910-07e8-4157-8120-59e269b7e503    584
588925ea-424f-455b-a126-ac18d32aa66a    567
60d33ac5-ee58-453b-a0d3-2f0b9166c1b5    567
64c13ccd-583e-4c7c-b0f8-afce34f33e15    618
6f8fb998-91fe-4668-9bfc-751deea7b2db    599
72731443-890b-49c2-8963-bb12923727f0    556
776ef556-3745-4b71-a056-4291

# Count of clicks in a window of time

If a user clicks too much (3 clicks or more in 2 seconds for example), it is not normal

In [36]:
#convert timestamps to windows

window_size = 2 #seconds
df_clicks['time_window'] = df_clicks['timestamp'] // window_size

In [37]:
#We confirm that this pattern is present

df_windows = df_clicks.groupby(['uid', 'time_window']).count()
df_windows[df_windows['eventType'] > 2]

eventType  timestamp  ip  \
uid                                  time_window                             
166aeaf4-9995-475a-8a50-f0ca9a34160e 812395645            3          3   3   
                                     812397093            3          3   3   
32b3f845-0d02-466c-95e7-a9177821541a 812394020            3          3   3   
                                     812394784            3          3   3   
3b72893e-916b-406d-9979-1f55c4a36678 812396574            3          3   3   
                                     812398115            3          3   3   
3bf12f77-5084-42b1-a9f4-5b59c8c3ba12 812397249            3          3   3   
469bbf16-fb45-4df2-9cac-bf0069980a2d 812398437            3          3   3   
48d35610-7b97-4ce6-9ca4-725c512409da 812397392            3          3   3   
4bce2c20-3fed-4af1-9177-542ee232ece2 812394621            3          3   3   
                                     812394850            3          3   3   
                                     812395902            3          3   3   
                                     812398724            3          3   3   
588925ea-424f-455b-a126-ac18d32aa66a 812395063            3          3   3   
64c13ccd-583e-4c7c-b0f8-afce34f33e15 812395394            3          3   3   
                                     812396966            3          3   3   
6f8fb998-91fe-4668-9bfc-751deea7b2db 812393832            3          3   3   
72731443-890b-49c2-8963-bb12923727f0 812398085            3          3   3   
815f86c3-eadf-4a3a-ad95-a945b1906a62 812395309            3          3   3   
                                     812397495            3          3   3   
9ffb6f65-7c1a-44e5-8f82-29f4d6baba16 812395583            3          3   3   
                                     812397531            3          3   3   
abed1237-40e2-4f24-a264-0d4819aa71fc 812398734            3          3   3   
b4dc943a-9130-476e-8ac4-4d122fb1ed74 812395638            3          3   3   
bce04c9a-6c46-41e3-a83f-f3e33f1c02ca 812396715            3          3   3   
c1339e58-ebbf-49c7-bb30-36e3b1832b8a 812395076            3          3   3   
c1d7d707-3d72-427a-8e04-bd3faabd4208 812394442            3          3   3   
c505e5f2-bf34-40ed-b9b5-d3185400b8a1 812394855            3          3   3   
ccf51780-df0a-41ea-a642-164dd3ea096c 812395482            3          3   3   
d6b9045c-046b-4718-bfec-44bce705ac4c 812394717            3          3   3   
dc18faca-c40c-4ec3-8ed6-e8540bf57d9a 812395806            3          3   3   
                                     812397260            3          3   3   
f4c34fdc-cc1b-4496-ae0c-1a77f5c078d2 812396448            3          3   3   
f881e9f3-d09f-45f7-9129-2753951be778 812394340            3          3   3   
                                     812397369            3          3   3   
                                     812397981            3          3   3   
ff095193-b2ee-47ba-84ef-b8d57eb6f49e 812394970            3          3   3   
                                     812396272            3          3   3   
                                     812397305            3          3   3   
                                     812397713            3          3   3   

                                                  impressionId  
uid                                  time_window                
166aeaf4-9995-475a-8a50-f0ca9a34160e 812395645               3  
                                     812397093               3  
32b3f845-0d02-466c-95e7-a9177821541a 812394020               3  
                                     812394784               3  
3b72893e-916b-406d-9979-1f55c4a36678 812396574               3  
                                     812398115               3  
3bf12f77-5084-42b1-a9f4-5b59c8c3ba12 812397249               3  
469bbf16-fb45-4df2-9cac-bf0069980a2d 812398437               3  
48d35610-7b97-4ce6-9ca4-725c512409da 812397392               3  
4bce2c20-3fed-4af1-9177-542ee232ece2 812394621            

# Looking for wierd Ips/ uids
Maybe some specific ip addresses/users are to be banned

In [45]:
# Max is 10200 uids, which is a lot
df_clicks_ips = df_clicks.groupby('ip').count()
df_clicks_ips.describe()

,eventType,uid,timestamp,impressionId,time_window
count,9782.000000,9782.000000,9782.000000,9782.000000,9782.000000
mean,4.998671,4.998671,4.998671,4.998671,4.998671
std,103.107669,103.107669,103.107669,103.107669,103.107669
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000
max,10200.000000,10200.000000,10200.000000,10200.000000,10200.000000


In [48]:
# We looked for "238.186.83.58" on https://www.iplocation.net/ and it corresponds to nowhere

df_clicks_ips[df_clicks_ips['eventType'] > 50]

,eventType,uid,timestamp,impressionId,time_window
ip,,,,,
238.186.83.58,10200,10200,10200,10200,10200


In [49]:
df_clicks_uids = df_clicks.groupby('uid').count()
df_clicks_uids.describe()

,eventType,timestamp,ip,impressionId,time_window
count,19030.000000,19030.000000,19030.000000,19030.000000,19030.000000
mean,2.569469,2.569469,2.569469,2.569469,2.569469
std,30.529604,30.529604,30.529604,30.529604,30.529604
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,649.000000,649.000000,649.000000,649.000000,649.000000


In [50]:
# There are many users who should be blacklisted. However, this does not seem to a an interestng filter to implement

df_clicks_uids[df_clicks_uids['eventType'] > 50]

,eventType,timestamp,ip,impressionId,time_window
uid,,,,,
01aa1f62-fd0c-40a1-a7cf-8e6adc1b4c95,639,639,639,639,639
07c38e89-9750-4727-8721-b80b5b03db4e,571,571,571,571,571
0d878df4-9966-43c8-aeb1-8c736bfbf772,624,624,624,624,624
10ffe29f-21b7-4f96-9540-91e77ee17d20,598,598,598,598,598
141a7059-895c-4f74-a002-d1b7e5e6cc85,598,598,598,598,598
166aeaf4-9995-475a-8a50-f0ca9a34160e,597,597,597,597,597
26c91786-7eee-46df-91cc-59eeae4350d2,631,631,631,631,631
32b3f845-0d02-466c-95e7-a9177821541a,592,592,592,592,592
3b72893e-916b-406d-9979-1f55c4a36678,625,625,625,625,625


In [55]:
# NB. some of the above patterns can also be found in displays

df_displays = pd.DataFrame(displays_content)
df_displays.head()

,eventType,uid,timestamp,ip,impressionId
0,display,9b87825e-15f0-4485-9784-e4ff972da756,1624787312,125.206.131.123,54c449d2-aa15-4e4a-8b17-06e8925755c6
1,display,cb76c5b8-301c-4b90-b136-367ab55b0f98,1624787312,176.117.247.114,26be660f-edd7-4c42-b79b-48b47ef24a07
2,display,8cfd4f50-f996-4c2c-a92d-46a0a3c806ba,1624787312,118.41.84.29,04c13a8b-a582-4669-9f83-d425f66f78f9
3,display,f51708ac-0b8c-4fe2-8cf4-3a92a428d2c8,1624787312,130.190.210.109,998109e6-c83a-4956-9e27-8c09c0207b14
4,display,9790cc60-9c83-41c8-b4e4-bc97a0a1c6ec,1624787312,227.168.202.73,55b688b6-4cbe-41f3-b218-107de4305e8a


In [56]:
print(len(df_clicks) / len(df_displays))

0.33747204814620546


In [ ]:
df_displays = pd.DataFrame(displays_content)
df_displays

In [ ]:
# df_displays_renamed = df_displays.rename(columns={'uid' : 'uid_display', 'ip' : 'ip_display', 'impressionId' : 'impressionId_display', 'timestamp' : 'timestamp_display'})
df_displays_renamed = df_displays.rename(columns={'ip' : 'ip_display', 'timestamp' : 'timestamp_display'})
df_displays_renamed = df_displays_renamed.drop('eventType', axis=1)

In [ ]:
df_joined = pd.merge(df_displays_renamed, df_clicks, on=['uid', 'impressionId'])
df_joined

In [ ]:
ids_df_joined = df_joined[['uid', 'impressionId']]
ids_df_joined
i2 = ids_df_joined.set_index(['uid', 'impressionId']).index
i1 = df_clicks.set_index(['uid', 'impressionId']).index

In [ ]:
df_clicks[~i1.isin(i2)]

In [ ]:
a = df_clicks[df_clicks['uid'] == '57e9e910-07e8-4157-8120-59e269b7e503']
b = a[a['impressionId'] == 'ab2cc313-6fa5-42fe-9622-efd33214b5b9']
b

In [ ]:
df1 = pd.DataFrame({'c': ['A', 'A', 'B', 'C', 'C'],
                    'k': [1, 2, 2, 2, 2],
                    'l': ['a', 'b', 'a', 'a', 'd']})
df2 = pd.DataFrame({'c': ['A', 'C'],
                    'l': ['b', 'a']})
keys = list(df2.columns.values)
i1 = df1.set_index(keys).index
i2 = df2.set_index(keys).index
df1[~i1.isin(i2)]

In [ ]:
a = df_clicks.groupby(['ip']).count()
a[a['eventType'] > 3]

In [ ]:
print(len(df_clicks['impressionId'])/ len(df_displays['impressionId']))

In [ ]:
df

In [ ]:
df = pd.concat([df_displays, df_clicks])

topic_df = pd.get_dummies(df['eventType'])
df = df.drop('eventType', axis=1)
final_df = df.join(topic_df)
final_df = final_df.groupby('impressionId').count()
final_df['click_through_rate'] = final_df['click'] / final_df['display']
final_df

In [ ]:
df = pd.concat([df_displays, df_clicks])
df.groupby(['impressionId', 'eventType']).count()

In [ ]:
from collections import Set

l= []
with open('too_many_ips', 'r') as f:
    for line in f:
        l.append(line.split(',')[1])